In [134]:
%matplotlib inline

import pandas as pd
import numpy as np
import networkx as nx
import json
from sklearn import metrics

In [135]:
def systematic_regular(no_of_samples, x, y):
    # Calculate the dimensions of the regular grid
    rows = int(np.sqrt(no_of_samples))
    cols = int(np.ceil(no_of_samples / rows))

    # Calculate the padding to ensure points are equally spaced
    padding_x = (x - (rows - 1)) // rows
    padding_y = (y - (cols - 1)) // cols

    # Initialize points list
    points = []

    # Select equally spaced points with padding
    for i in range(rows):
        for j in range(cols):
            points.append((int((i * (padding_x + 1)) + ((padding_x + 1)/2)), int((j * (padding_y + 1) + ((padding_y + 1)/2)))))

    return points[:no_of_samples]

def wageningen_w():
    return [(25, 25), (25, 75), (25, 125), (25, 175), (63, 75), (88, 125), (138, 75), (113, 125), (175, 25), (175, 75), (175, 125), (175, 175)]
    
def retrieve_ids_per_sample_von_neumann(points, data, r):
    samples =  [ [] for _ in range(len(points)) ]
    for ind, row in data.iterrows():
        for index, point in enumerate(points):
            x1, y1 = point
            x2 = row.x
            y2 = row.y

            if (abs(x1 - x2) + abs(y1 - y2)) <= r:
                samples[index].append(ind)
    return samples

def pearson_cooccurrence(abundance_data, threshold=0):
    # Initialize a graph
    G = nx.Graph()
    G.add_nodes_from(range(9))

    # Calculate the Pearson correlation coefficient matrix
    pearson_matrix = np.corrcoef(abundance_data, rowvar=False)
    
    pearson_matrix[np.isnan(pearson_matrix)] = 0

    # Get the number of entities
    num_entities = len(abundance_data[0])

    # Iterate through the Pearson correlation matrix to create co-occurrence edges
    for i in range(num_entities):
        for j in range(i + 1, num_entities):
            # Check if the correlation coefficient exceeds the threshold
            if (pearson_matrix[i][j] > threshold):
                G.add_edge(i, j, weight=pearson_matrix[i][j])

    return G, pearson_matrix

def manhattan_neighborhood_matrix(df):
    # Initialize the grid dictionary
    grid = {}
    
    # Populate the grid with entity locations and types
    for idx, row in df.iterrows():
        t, x, y, z = row['type'], row['x'], row['y'], row['z']
        grid[(x, y, z)] = t
    
    # Define Manhattan neighbors offsets (distance 1 in 3D)
    neighbors_offsets = [
        (dx, dy, dz) 
        for dx in (-1, 0, 1) 
        for dy in (-1, 0, 1) 
        for dz in (-1, 0, 1)
        if dx != 0 or dy != 0 or dz != 0
    ]
    
    # Initialize the 9x9 matrix
    matrix = np.zeros((9, 9), dtype=int)
    
    # Calculate type counts in the neighborhood
    for (x, y, z), t in grid.items():
        for dx, dy, dz in neighbors_offsets:
            neighbor_pos = (x + dx, y + dy, z + dz)
            if neighbor_pos in grid:
                neighbor_type = grid[neighbor_pos]
                matrix[int(t)][int(neighbor_type)] += 1
    
    return matrix

In [136]:
filenames = ['agent_log_0', 'agent_log_1','agent_log_2','agent_log_3', 'agent_log_4', 'agent_log_5', 'agent_log_6', 'agent_log_7', 'agent_log_8', 'agent_log_9', 'agent_log_10', 'agent_log_11', 'agent_log_12', 'agent_log_13', 'agent_log_14', 'agent_log_15', 'agent_log_16', 'agent_log_17', 'agent_log_18', 'agent_log_19', 'agent_log_20']
rs = [1, 3, 5, 7]
inits = [0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6]

#filenames = ['agent_log_20']
#rs = [3]
#inits = [6]

x_max = 200
y_max = 200
z_max = 25

no_of_samples = 16

selected_points_reg = systematic_regular(no_of_samples, x_max, y_max)
selected_points_w = wageningen_w()

Gs = []

Gs.append(nx.node_link_graph(json.loads('{"directed": false, "multigraph": false, "graph": {}, "nodes": [{"id": 0}, {"id": 1}, {"id": 2}, {"id": 3}, {"id": 4}, {"id": 5}, {"id": 6}, {"id": 7}, {"id": 8}], "links": [{"weight": 0.5826312884833548, "source": 0, "target": 3}, {"weight": 0.6148356427539814, "source": 0, "target": 4}, {"weight": 0.7306576990838498, "source": 1, "target": 2}, {"weight": 0.9003900420385831, "source": 1, "target": 5}, {"weight": -0.638579754693958, "source": 1, "target": 7}, {"weight": -0.27796656418881527, "source": 2, "target": 7}, {"weight": 0.46123412849139406, "source": 3, "target": 4}, {"weight": 0.6068144174853789, "source": 5, "target": 8}]}')))
Gs.append(nx.node_link_graph(json.loads('{"directed": false, "multigraph": false, "graph": {}, "nodes": [{"id": 0}, {"id": 1}, {"id": 2}, {"id": 3}, {"id": 4}, {"id": 5}, {"id": 6}, {"id": 7}, {"id": 8}], "links": [{"weight": 0.5826312884833548, "source": 0, "target": 2}, {"weight": 0.6148356427539814, "source": 0, "target": 8}, {"weight": 0.7306576990838498, "source": 2, "target": 4}, {"weight": 0.9003900420385831, "source": 2, "target": 8}, {"weight": -0.638579754693958, "source": 3, "target": 6}, {"weight": -0.27796656418881527, "source": 3, "target": 8}, {"weight": 0.46123412849139406, "source": 4, "target": 5}, {"weight": 0.6068144174853789, "source": 7, "target": 8}]}')))
Gs.append(nx.node_link_graph(json.loads('{"directed": false, "multigraph": false, "graph": {}, "nodes": [{"id": 0}, {"id": 1}, {"id": 2}, {"id": 3}, {"id": 4}, {"id": 5}, {"id": 6}, {"id": 7}, {"id": 8}], "links": [{"weight": 0.5826312884833548, "source": 1, "target": 4}, {"weight": 0.6148356427539814, "source": 1, "target": 5}, {"weight": 0.7306576990838498, "source": 1, "target": 6}, {"weight": 0.9003900420385831, "source": 2, "target": 5}, {"weight": -0.638579754693958, "source": 2, "target": 6}, {"weight": -0.27796656418881527, "source": 2, "target": 8}, {"weight": 0.46123412849139406, "source": 3, "target": 5}, {"weight": 0.6068144174853789, "source": 4, "target": 5}, {"weight": 0.5112680070896285, "source": 4, "target": 6}, {"weight": 0.42525572507758125, "source": 4, "target": 7}, {"weight": 0.4601658168250095, "source": 4, "target": 8}, {"weight": 0.4943223393958548, "source": 5, "target": 6}]}')))
Gs.append(nx.node_link_graph(json.loads('{"directed": false, "multigraph": false, "graph": {}, "nodes": [{"id": 0}, {"id": 1}, {"id": 2}, {"id": 3}, {"id": 4}, {"id": 5}, {"id": 6}, {"id": 7}, {"id": 8}], "links": [{"weight": 0.5826312884833548, "source": 0, "target": 1}, {"weight": 0.6148356427539814, "source": 0, "target": 3}, {"weight": 0.7306576990838498, "source": 0, "target": 5}, {"weight": 0.9003900420385831, "source": 0, "target": 8}, {"weight": -0.638579754693958, "source": 1, "target": 3}, {"weight": -0.27796656418881527, "source": 1, "target": 5}, {"weight": 0.46123412849139406, "source": 1, "target": 6}, {"weight": 0.6068144174853789, "source": 3, "target": 7}, {"weight": 0.5112680070896285, "source": 3, "target": 8}, {"weight": 0.42525572507758125, "source": 4, "target": 5}, {"weight": 0.4601658168250095, "source": 4, "target": 6}, {"weight": 0.4943223393958548, "source": 4, "target": 7}, {"weight": -0.14671525430854027, "source": 4, "target": 8}]}')))
Gs.append(nx.node_link_graph(json.loads('{"directed": false, "multigraph": false, "graph": {}, "nodes": [{"id": 0}, {"id": 1}, {"id": 2}, {"id": 3}, {"id": 4}, {"id": 5}, {"id": 6}, {"id": 7}, {"id": 8}], "links": [{"weight": 0.5826312884833548, "source": 0, "target": 1}, {"weight": 0.6148356427539814, "source": 0, "target": 3}, {"weight": 0.7306576990838498, "source": 0, "target": 4}, {"weight": 0.9003900420385831, "source": 0, "target": 6}, {"weight": -0.638579754693958, "source": 0, "target": 7}, {"weight": -0.27796656418881527, "source": 1, "target": 3}, {"weight": 0.46123412849139406, "source": 1, "target": 4}, {"weight": 0.6068144174853789, "source": 1, "target": 6}, {"weight": 0.5112680070896285, "source": 1, "target": 7}, {"weight": 0.42525572507758125, "source": 1, "target": 8}, {"weight": 0.4601658168250095, "source": 2, "target": 5}, {"weight": 0.4943223393958548, "source": 2, "target": 6}, {"weight": -0.14671525430854027, "source": 2, "target": 8}, {"weight": -0.4038872117981791, "source": 3, "target": 4}, {"weight": 0.5176903249517476, "source": 3, "target": 6}, {"weight": 0.7534244447856359, "source": 3, "target": 7}, {"weight": 0.5244768288592718, "source": 4, "target": 7}, {"weight": 0.7363358287149884, "source": 4, "target": 8}, {"weight": 0.631262017598583, "source": 5, "target": 6}, {"weight": 0.13566419265027413, "source": 5, "target": 8}, {"weight": 0.7444351489453024, "source": 6, "target": 7}, {"weight": -0.4320948133020824, "source": 7, "target": 8}]}')))
Gs.append(nx.node_link_graph(json.loads('{"directed": false, "multigraph": false, "graph": {}, "nodes": [{"id": 0}, {"id": 1}, {"id": 2}, {"id": 3}, {"id": 4}, {"id": 5}, {"id": 6}, {"id": 7}, {"id": 8}], "links": [{"weight": 0.5826312884833548, "source": 0, "target": 1}, {"weight": 0.6148356427539814, "source": 0, "target": 3}, {"weight": 0.7306576990838498, "source": 0, "target": 4}, {"weight": 0.9003900420385831, "source": 0, "target": 6}, {"weight": -0.638579754693958, "source": 0, "target": 7}, {"weight": -0.27796656418881527, "source": 0, "target": 8}, {"weight": 0.46123412849139406, "source": 1, "target": 2}, {"weight": 0.6068144174853789, "source": 1, "target": 4}, {"weight": 0.5112680070896285, "source": 1, "target": 5}, {"weight": 0.42525572507758125, "source": 1, "target": 6}, {"weight": 0.4601658168250095, "source": 1, "target": 7}, {"weight": 0.4943223393958548, "source": 2, "target": 4}, {"weight": -0.14671525430854027, "source": 2, "target": 5}, {"weight": -0.4038872117981791, "source": 3, "target": 4}, {"weight": 0.5176903249517476, "source": 3, "target": 5}, {"weight": 0.7534244447856359, "source": 3, "target": 6}, {"weight": 0.5244768288592718, "source": 3, "target": 8}, {"weight": 0.7363358287149884, "source": 4, "target": 7}, {"weight": 0.631262017598583, "source": 4, "target": 8}, {"weight": 0.13566419265027413, "source": 5, "target": 7}, {"weight": 0.7444351489453024, "source": 5, "target": 8}, {"weight": -0.4320948133020824, "source": 6, "target": 8}, {"weight": -0.581091373256546, "source": 7, "target": 8}]}')))
Gs.append(nx.node_link_graph(json.loads('{"directed": false, "multigraph": false, "graph": {}, "nodes": [{"id": 0}, {"id": 1}, {"id": 2}, {"id": 3}, {"id": 4}, {"id": 5}, {"id": 6}, {"id": 7}, {"id": 8}], "links": [{"weight": 0.5826312884833548, "source": 0, "target": 5}, {"weight": 0.6148356427539814, "source": 0, "target": 8}, {"weight": 0.7306576990838498, "source": 1, "target": 3}, {"weight": 0.9003900420385831, "source": 1, "target": 4}, {"weight": -0.638579754693958, "source": 1, "target": 6}, {"weight": -0.27796656418881527, "source": 2, "target": 4}, {"weight": 0.46123412849139406, "source": 2, "target": 7}, {"weight": 0.6068144174853789, "source": 2, "target": 8}, {"weight": 0.5112680070896285, "source": 3, "target": 4}, {"weight": 0.42525572507758125, "source": 3, "target": 6}, {"weight": 0.4601658168250095, "source": 3, "target": 7}, {"weight": 0.4943223393958548, "source": 3, "target": 8}, {"weight": -0.14671525430854027, "source": 4, "target": 5}, {"weight": -0.4038872117981791, "source": 4, "target": 6}, {"weight": 0.5176903249517476, "source": 5, "target": 7}, {"weight": 0.7534244447856359, "source": 5, "target": 8}]}')))

In [137]:
# File path where you want to write the lines
file_path = "analysis_results.txt"

for idx, filename in enumerate(filenames):
    G = Gs[inits[idx]]
    A = nx.adjacency_matrix(G).todense()
    
    df = pd.read_csv("../sosi/outputs/" + filename + ".csv")
    data = df[df["tick"] == 50]
    data = data.reset_index(drop=True)
    
    for r in rs:
        samples_w = retrieve_ids_per_sample_von_neumann(selected_points_w, data, r)
        samples_reg = retrieve_ids_per_sample_von_neumann(selected_points_reg, data, r)
    
        sample_site_counts_w = []
        for sample_site in samples_w:
            df2 = data.iloc[sample_site]
            count = df2['type'].value_counts().reindex(range(len(df["type"].unique())), fill_value=0)
            sample_site_counts_w.append([val for _, val in count.items()])
    
        sample_site_counts_reg = []
        for sample_site in samples_reg:
            df2 = data.iloc[sample_site]
            count = df2['type'].value_counts().reindex(range(len(df["type"].unique())), fill_value=0)
            sample_site_counts_reg.append([val for _, val in count.items()])
            
        neighborhood_counts = manhattan_neighborhood_matrix(data)
    
        gt_pearson_net, gt_pearson = pearson_cooccurrence(neighborhood_counts, 0)
        pearson_net_w, pearson_w = pearson_cooccurrence(sample_site_counts_w, 0)
        pearson_net_reg, pearson_reg = pearson_cooccurrence(sample_site_counts_reg, 0)
        
        lines = []
        
        lines.append(filename + " Diameter: " + str(r))
        lines.append("Initial co-occurrence <-> Ground truth")
        lines.append("MAE: " + str(metrics.mean_absolute_error(A, gt_pearson)))
        lines.append("MdAE: " + str(metrics.median_absolute_error(A, gt_pearson)))
        lines.append("R2: " + str(metrics.r2_score(A, gt_pearson)))
        lines.append("Ground truth <-> Wageningen W")
        lines.append("MAE: " + str(metrics.mean_absolute_error(gt_pearson, pearson_w)))
        lines.append("MdAE: " + str(metrics.median_absolute_error(gt_pearson, pearson_w)))
        lines.append("R2: " + str(metrics.r2_score(gt_pearson, pearson_w)))
        lines.append("Initial co-occurrence <-> Wageningen W")
        lines.append("MAE: " + str(metrics.mean_absolute_error(A, pearson_w)))
        lines.append("MdAE: " + str(metrics.median_absolute_error(A, pearson_w)))
        lines.append("R2: " + str(metrics.r2_score(A, pearson_w)))
        lines.append("Ground truth <-> Sys Reg")
        lines.append("MAE: " + str(metrics.mean_absolute_error(gt_pearson, pearson_reg)))
        lines.append("MdAE: " + str(metrics.median_absolute_error(gt_pearson, pearson_reg)))
        lines.append("R2: " + str(metrics.r2_score(gt_pearson, pearson_reg)))
        lines.append("Initial co-occurrence <-> Sys Reg")
        lines.append("MAE: " + str(metrics.mean_absolute_error(A, pearson_reg)))
        lines.append("MdAE: " + str(metrics.median_absolute_error(A, pearson_reg)))
        lines.append("R2: " + str(metrics.r2_score(A, pearson_reg)) + "\n")
        
        # Open the file in write mode
        with open(file_path, "w") as file:
            # Write each line to the file
            for line in lines:
                file.write(line + "\n")